In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.layers import Dense, Dropout

Using TensorFlow backend.


In [2]:
from google.colab import files
src = list(files.upload().values())[0]
open('resnet_builder.py','wb').write(src)
from resnet_builder import resnet

Saving resnet_builder.py to resnet_builder (8).py


In [3]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

x_train = x_train / 255.
x_test = x_test / 255.
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [0]:
batch_size = 128 
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 50 # 訓練的 epochs 數量

In [5]:
# 建立 ResNet 模型
model = resnet(input_shape = (32, 32, 3), num_classes = num_classes) 
model.summary()

model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(),
              metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
_____________________________________

In [0]:
from keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator( 
                                    rotation_range = 20, 
                                    width_shift_range = 0.2,
                                    height_shift_range = 0.2,
                                    horizontal_flip = True
                                    )

from keras.callbacks import EarlyStopping
earlystop = EarlyStopping(monitor = "val_loss", patience = 10, verbose = 1 )

from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(factor = 0.3, min_lr = 1e-12, monitor = 'val_loss', patience = 5, verbose = 1)

In [7]:
model.fit_generator(
                    data_generator.flow(x_train, y_train, batch_size = batch_size),
                    steps_per_epoch = len(x_train) / batch_size, epochs = epochs,
                    callbacks = [earlystop, reduce_lr],
                    )

Instructions for updating:
Use tf.cast instead.
Epoch 1/50
391/390 [==============================] - 42s 108ms/step - loss: 2.0293 - acc: 0.4486
Epoch 2/50
  1/390 [..............................] - ETA: 31s - loss: 1.7139 - acc: 0.5859

/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:569: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1109: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


391/390 [==============================] - 34s 87ms/step - loss: 1.6007 - acc: 0.5704
Epoch 3/50
391/390 [==============================] - 34s 88ms/step - loss: 1.4049 - acc: 0.6258
Epoch 4/50
391/390 [==============================] - 34s 88ms/step - loss: 1.2841 - acc: 0.6593
Epoch 5/50
391/390 [==============================] - 35s 89ms/step - loss: 1.1935 - acc: 0.6861
Epoch 6/50
391/390 [==============================] - 35s 89ms/step - loss: 1.1166 - acc: 0.7119
Epoch 7/50
391/390 [==============================] - 35s 90ms/step - loss: 1.0568 - acc: 0.7286
Epoch 8/50
391/390 [==============================] - 35s 88ms/step - loss: 1.0012 - acc: 0.7500
Epoch 9/50
391/390 [==============================] - 35s 89ms/step - loss: 0.9554 - acc: 0.7641
Epoch 10/50
391/390 [==============================] - 35s 90ms/step - loss: 0.9201 - acc: 0.7755
Epoch 11/50
391/390 [==============================] - 35s 88ms/step - loss: 0.8973 - acc: 0.7786
Epoch 12/50
391/390 [==================

In [8]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.7838085087776184
Test accuracy: 0.8336
